<a href="https://colab.research.google.com/github/hrbolek/func2pipe/blob/master/notebooks/func2pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Func to Pipe

## Source Code

### Functions Reducer

In [0]:
def createPipe(*F):
  def Fdiamond(sequence):
    result = sequence
    for Fx in F:
      result = Fx(result)
    return result
  return Fdiamond

### Subpipe

In [0]:
def createSub(assign = lambda source, result: {**source, **result}, reducer = None):

    def multipleResults(*F):
        buffer = {'data': None}
        def pre(gen):
            for item in gen:
                buffer['data'] = item
                yield item

        def inner(gen):
            e = gen
            for f in F:
                e = f(e)
            return e

        def post(gen):
            for item in gen:
                yield assign(buffer['data'], item)

        return lambda gen: post(inner(pre(gen)))

    def reducedResult(*F):
        def inner(gen):
            e = gen
            for f in F:
                e = f(e)
            return e

        def result(gen):
            for item in gen:
                itemResult = reducer(inner([item]))
                yield assign(item, itemResult)
        return result

    if reducer == None:
        return multipleResults
    else:
        return reducedResult

### Convertor

In [0]:
from functools import wraps, reduce, partial


In [0]:
from functools import wraps, reduce, partial
def convertToPipeFuncFull(func, with_yield = False, with_state = False, kwargs = {}):
    @wraps(func)
    def innerSelectSimple(generator):
        for i in generator:
            yield func(i, **kwargs)

    if (not with_yield) & (not with_state):
        return innerSelectSimple
    
    @wraps(func)
    def innerSelectWithYield(generator):
        for i in generator:
            for j in func(i, **kwargs):
                yield j


    if (with_yield) & (not with_state):
        return innerSelectWithYield

    @wraps(func)
    def innerSelectWithState(generator):
        state = None
        initState = True
        for i in generator:
            result = None
            if (initState):
                result, state = func(i, **kwargs)
                initState = False
            else:
                result, state = func(i, state = state, **kwargs)
            if result:
              yield result

    if (not with_yield) & with_state:
        return innerSelectWithState

    def innerSelectWithYieldWithState(generator):
        state = None
        initState = True
        for i in generator:
            result = None
            if (initState):
                for result, statel in func(i, **kwargs):
                    state = statel
                    if result:
                        yield result
                initState = False
            else:
                for result, statel in func(i, state = state, **kwargs):
                    state = statel
                    if result:
                        yield result


    return innerSelectWithYieldWithState

### Convertor Decorator

In [0]:
def convertToPipeFunc(with_yield = False, with_state = False):
    def pipeit(__func):
        @wraps(__func)
        def binder(**kwargs):
            result = wraps(__func)(convertToPipeFuncFull(__func, with_yield, with_state, kwargs))
            return result
        return binder
    return pipeit

convertToPipeFunc2 = convertToPipeFunc 

## Tests

In [0]:
import unittest

class TestCase(unittest.TestCase):
  def __call__(self, *args, **kwargs):
    self.assertEqual(*args, **kwargs)
    return True

testerClass = TestCase()

### Simple Function

In [62]:
source = [{'value': 0}, {'value': 1}, {'value': 2}]
@convertToPipeFunc()
def add(item, amount):
  return {**item, 'result': item['value'] + amount}

expectedResult = [{'value': 0, 'result': 3}, {'value': 1, 'result': 4},
  {'value': 2, 'result': 5}]

pipe = createPipe(
    add(amount = 2),
    add(amount = 3),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testerClass(expectedResult, result)) 

Source: [{'value': 0}, {'value': 1}, {'value': 2}]
Result: [{'value': 0, 'result': 3}, {'value': 1, 'result': 4}, {'value': 2, 'result': 5}]
Valid:  True


### Function with Yield

In [63]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@convertToPipeFunc(with_yield = True)
def revealSubItem(item, itemName):
    for _ in item[itemName]:
        yield _

expectedResult = ['A', 'B', 'A', 'C', 'D', 'E']

pipe = createPipe(
    revealSubItem(itemName = 'value'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testerClass(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: ['A', 'B', 'A', 'C', 'D', 'E']
Valid:  True


### Function with State

In [64]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@convertToPipeFunc(with_state = True)
def assignId(item, state = 0, idName = 'id'):
    return {**item, idName: state}, state + 1

expectedResult = [{'value': ['A', 'B'], 'ID': 0}, 
    {'value': ['A', 'C'], 'ID': 1}, {'value': ['D', 'E'], 'ID': 2}]

pipe = createPipe(
    assignId(idName = 'ID'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testerClass(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: [{'value': ['A', 'B'], 'ID': 0}, {'value': ['A', 'C'], 'ID': 1}, {'value': ['D', 'E'], 'ID': 2}]
Valid:  True


### Function with Yield and with State

In [69]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@convertToPipeFunc(with_yield = True, with_state = True)
def assignIdToSubItem(item, state = -1, itemName = ''):
    for _ in item[itemName]:
        state = state + 1
        yield {itemName: _, 'id': state}, state

expectedResult = [{'value': 'A', 'id': 0}, {'value': 'B', 'id': 1}, 
  {'value': 'A', 'id': 2}, {'value': 'C', 'id': 3}, 
  {'value': 'D', 'id': 4}, {'value': 'E', 'id': 5}
  ]

pipe = createPipe(
    assignIdToSubItem(itemName = 'value'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testerClass(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: [{'value': 'A', 'id': 0}, {'value': 'B', 'id': 1}, {'value': 'A', 'id': 2}, {'value': 'C', 'id': 3}, {'value': 'D', 'id': 4}, {'value': 'E', 'id': 5}]
Valid:  True


In [66]:
source = [{'value': 0}, {'value': 1}, {'value': 2}]
@convertToPipeFunc()
def add(item, amount):
  return {**item, 'result': item['value'] + amount}

@convertToPipeFunc()
def calc(item, func, name):
  result = {}
  result[name] = func(item)
  return result

@convertToPipeFunc(with_yield = True)
def extra(item, extradata):
  for _ in extradata:
    yield _

def createReducer(accumulator = [], f = lambda item, accumulator: [*accumulator, item]):
  def reducer(gen):
    result = accumulator
    for item in gen:
      result = f(item, result)
    return result
  return reducer

pipe_01 = createPipe(
    add(amount = 2),
    createSub(assign = lambda source, result: {**source, 'value2': result}, reducer = createReducer())(
#    SUB(assign = lambda source, result: {**source, **result})(
#    SUB(assign = lambda source, result: {**source, 'value2': result})(
      calc(func = lambda item: 4, name = 'extraValue'),
      extra(extradata = ['A', 'B', 'C'])
    )
    )
result = list(pipe_01(source))
print(result)


[{'value': 0, 'result': 2, 'value2': ['A', 'B', 'C']}, {'value': 1, 'result': 3, 'value2': ['A', 'B', 'C']}, {'value': 2, 'result': 4, 'value2': ['A', 'B', 'C']}]


In [67]:

greatSource = [{'id': 'A'}, {'id': 'B'}, {'id': 'C'}]
smallSource = [{'id': 'a'}, {'id': 'b'}, {'id': 'c'}]
fromGreat2Small = {'A': 'a', 'B': 'b', 'C': 'c'}
fromSmall2Great = {'a': 'A', 'b': 'B', 'c': 'C'}

@convertToPipeFunc2()
def greatToSmallRelation(greatItem):
  print('G:', greatItem)
  return {'idS': fromGreat2Small[greatItem['id']]}

@convertToPipeFunc2()
def smallToGreatRelation(smallItem):
  #print('S:', smallItem)
  return {'id': fromSmall2Great[smallItem['id']]}

@convertToPipeFunc2(with_yield=True)
def smallToGreatRelation2(smallItem):
  #print('S:', smallItem)
  for i in greatSource:
    yield i
  #return {'idG': fromSmall2Great[smallItem['id']]}

@convertToPipeFunc2()
def identityPipe(item, description):
  return {**item, 'description': description}

@convertToPipeFunc2(with_yield=True)
def anyToNumbersRelation(item):
  for i in range(1,3):
    #yield {'value': i}
    yield i

@convertToPipeFunc2()
def numberRecord(item):
  return {'value': item}

@convertToPipeFunc2()
def printItem(item, description):
  print(description, item)
  return item

gr = {
    'nodes': {
        'Great': identityPipe(description = 'Great item'), 
        'Small': identityPipe(description = 'Small item'),
        'Numbers': numberRecord(),
        },
    'edges': [
        {'from': 'Great', 'to': 'Small', 'relation': greatToSmallRelation(), 'itemname' : 'little'},
        {'from': 'Small', 'to': 'Great', 'relation': smallToGreatRelation2(), 'itemname' : 'bigBC', 'filter': lambda item: item[id] > 'A'},
        {'from': 'Small', 'to': 'Great', 'relation': smallToGreatRelation2(), 'itemname' : 'bigA', 'filter': lambda item: item[id] <= 'A'},
        {'from': 'Small', 'to': 'Numbers', 'relation': anyToNumbersRelation(), 'itemname' : 'numbers'},
        {'from': 'Great', 'to': 'Numbers', 'relation': anyToNumbersRelation(), 'itemname' : 'numbers',
         'reducers' :[
            {'name': 'count', 
             'selector': lambda item: item['value'], 
             'reducer': lambda data: reduce(lambda x, y: (x + 1), data, 0)}]},
    ]
    }

def createArrayReducer(accumulator = [], f = lambda accumulator, item: [*accumulator, item]):
  def reducer(gen):
    result = accumulator
    for item in gen:
      result = f(result, item)
    return result
  return reducer

def createArrayReducer2(accumulator = [], f = lambda accumulator, item: [*accumulator, item]):
  result = lambda gen: reduce(f, gen, accumulator)
  return result


def builder(graph, currentnode):
  availableNodes = []
  for node in graph['nodes']:
    availableNodes.append(node)
  availableNodes.remove(currentnode)

  def innerBuilder(graph, currentnode, filterq = lambda item: True):
      descriptorPipe = graph['nodes'][currentnode]

      def x(relation):
        itemname = relation['to']
        if ('itemname' in relation):
            itemname = relation['itemname']
        filterq = lambda item: True
        if ('filter' in relation):
            filterq = relation['filter']

        innerPipe = innerBuilder(graph, relation['to'], filterq)

        sub = createSub(assign = lambda item, result: {**item, itemname: result}, reducer = createArrayReducer())
        return sub(relation['relation'], innerPipe)

      result = [descriptorPipe]
      nodesToRemove = []
      for relation in filter(lambda item: item['from'] == currentnode, graph['edges']):
          nodeName = relation['to']
          nodesToRemove.append(nodeName)
          if nodeName in availableNodes:
            #result += [printItem(description = 'pre'), (x(relation)), printItem(description = 'pos')]
            result += [x(relation)]
          
      for node in nodesToRemove:
        if node in availableNodes:
          availableNodes.remove(node)

      return createPipe(*result)
  return innerBuilder(graph, currentnode)

import json
graphPipe = builder(gr, 'Small')
result = list(graphPipe(smallSource))
print(json.dumps(result, indent=2))

[
  {
    "id": "a",
    "description": "Small item",
    "bigBC": [
      {
        "id": "A",
        "description": "Great item",
        "numbers": [
          {
            "value": 1
          },
          {
            "value": 2
          }
        ]
      },
      {
        "id": "B",
        "description": "Great item",
        "numbers": [
          {
            "value": 1
          },
          {
            "value": 2
          }
        ]
      },
      {
        "id": "C",
        "description": "Great item",
        "numbers": [
          {
            "value": 1
          },
          {
            "value": 2
          }
        ]
      }
    ],
    "bigA": [
      {
        "id": "A",
        "description": "Great item"
      },
      {
        "id": "B",
        "description": "Great item"
      },
      {
        "id": "C",
        "description": "Great item"
      }
    ]
  },
  {
    "id": "b",
    "description": "Small item",
    "bigBC": [
      {
        "id":

In [68]:
@convertToPipeFunc2()
def copy(item, description):
    """describes and converts item into dictionary."""
    return {'input': item, 'description': description}

@convertToPipeFunc2(with_yield = True)
def copyY(item, description):
    """describes and converts item into dictionary."""
    yield {'input': item, 'description': description}

@convertToPipeFunc2(with_yield = False, with_state = True)
def copyS(item, description, state = 0):
    """describes and converts item into dictionary."""
    return ({'input': item, 'description': description, 'state': state}, state+1)

@convertToPipeFunc2(with_yield = True, with_state = True)
def copyYS(item, description, state = 0):
    """describes and converts item into dictionary."""
    yield ({'input': item, 'description': description, 'state': state}, state+1)



print(copy.__doc__)
print(copyY.__doc__)
print(newcopyY.__doc__)
print(newcopyY2.__doc__)

data = [1, 2, 3]
#%%time
print(list(newcopy(data)))
print(list(newcopy2(data)))
print(list(newcopyY(data)))
print(list(newcopyY2(data)))
print(list(copyS(description = 'desc')(data)))
print(list(copyYS(description = 'desc')(data)))
data = range(0, 100000)
%timeit (list(newcopy(data)))
%timeit (list(newcopy2(data)))


describes and converts item into dictionary.
describes and converts item into dictionary.


NameError: ignored